In [52]:
#from dicom_contour.contour import *
import matplotlib.pyplot as plt
import numpy as np
import glob,os
import pydicom as dicom
from scipy.sparse import csc_matrix
from collections import defaultdict
import shutil
import operator
import warnings

In [2]:
path = './Breast_MRI_cont/Joly/'

In [28]:
#adapted from dicom_contour package. see dicom-contour on github.

def get_contour_file(path):
    """
    Get contour file from a given path by searching for ROIContourSequence 
    inside dicom data structure.
    More information on ROIContourSequence available here:
    http://dicom.nema.org/medical/dicom/2016c/output/chtml/part03/sect_C.8.8.6.html
    
    Inputs:
            path (str): path of the the directory that has DICOM files in it, e.g. folder of a single patient
    Return:
        contour_file (str): name of the file with the contour
    """
    # handle `/` missing
    if path[-1] != '/': path += '/'
    # get .dcm contour file
    fpaths = [path + f for f in os.listdir(path) if '.dcm' in f]
    n = 0
    for fpath in fpaths:
        f = dicom.read_file(fpath)
        if 'ROIContourSequence' in dir(f):
            contour_file = fpath.split('/')[-1]
            n += 1
    if n > 1: warnings.warn("There are multiple files, returning the last one!")
    return contour_file
def get_roi_names(contour_data):
    """
    This function will return the names of different contour data, 
    e.g. different contours from different experts and returns the name of each.
    Inputs:
        contour_data (dicom.dataset.FileDataset): contour dataset, read by dicom.read_file
    Returns:
        roi_seq_names (list): names of the 
    """
    roi_seq_names = [roi_seq.ROIName for roi_seq in list(contour_data.StructureSetROISequence)]
    return roi_seq_names

In [82]:
#get the file name
contour_file = get_contour_file(path)

print(isinstance(contour_file, file))

#store its data
contour_data = dicom.read_file(path + '/' + contour_file)

get_roi_names(contour_data)

False


['Organ', 'GTV']

In [83]:
def coord2pixels(contour_dataset, path):
    """
    Given a contour dataset (a DICOM class) and path that has .dcm files of
    corresponding images. This function will return img_arr and contour_arr (2d image and contour pixels)
    Inputs
        contour_dataset: DICOM dataset class that is identified as (3006, 0016)  Contour Image Sequence
        path: string that tells the path of all DICOM images
    Return
        img_arr: 2d np.array of image with pixel intensities
        contour_arr: 2d np.array of contour with 0 and 1 labels
    """

    #ContourData is the pixel data 
    contour_coord = contour_dataset.ContourData
    
    # x, y, z coordinates of the contour in mm
    coord = []
    for i in range(0, len(contour_coord), 3):
        coord.append((contour_coord[i], contour_coord[i + 1], contour_coord[i + 2]))

    # extract the image id corresponding to given countour
    # read that dicom file
    img_ID = contour_dataset.ContourImageSequence[0].ReferencedSOPInstanceUID
    img = dicom.read_file(path + img_ID + '.dcm')
    img_arr = img.pixel_array

    # physical distance between the center of each pixel
    x_spacing, y_spacing = float(img.PixelSpacing[0]), float(img.PixelSpacing[1])

    # this is the center of the upper left voxel
    origin_x, origin_y, _ = img.ImagePositionPatient

    # y, x is how it's mapped
    pixel_coords = [(np.ceil((y - origin_y) / y_spacing), np.ceil((x - origin_x) / x_spacing)) for x, y, _ in coord]

    # get contour data for the image
    rows = []
    cols = []
    for i, j in list(set(pixel_coords)):
        rows.append(i)
        cols.append(j)
    contour_arr = csc_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int8, shape=(img_arr.shape[0],img_arr.shape[1])).toarray()

    return img_arr, contour_arr, img_ID
def cfile2pixels(file, path, ROIContourSeq=0):
    """
    Given a contour file and path of related images return pixel arrays for contours
    and their corresponding images.
    Inputs
        file: filename of contour
        path: path that has contour and image files
        ROIContourSeq: tells which sequence of contouring to use default 0 (RTV)
    Return
        contour_iamge_arrays: A list which have pairs of img_arr and contour_arr for a given contour file
    """
    # handle `/` missing
    if path[-1] != '/': path += '/'
    f = dicom.read_file(path + file)
    # index 0 means that we are getting RTV information
    RTV = f.ROIContourSequence[ROIContourSeq]
    
    #the file doesnt have 'MR.' infront of it so manually added
    path+='MR.'
    
    # get contour datasets in a list
    contours = [contour for contour in RTV.ContourSequence]
    img_contour_arrays = [coord2pixels(cdata, path) for cdata in contours]
    return img_contour_arrays

In [84]:
# get all image-contour array pairs
contour_arrays = cfile2pixels(file=contour_file, path=path, ROIContourSeq=1)

ValueError: negative row index found

In [127]:
print(path)
print(contour_file)
f = dicom.read_file(path+contour_file)

RTV = f.ROIContourSequence[ROIContourSeq]
contours = [contour for contour in RTV.ContourSequence]

print(len(contours))
print('\n \n Countours[0]= ----------------------------------------')
print(contours[0])
print('----------------------------------------\n \n ')

contour_dataset = contours[0]
contour_coord = contour_dataset.ContourData
path = path

coord = []

for i in range(0, len(contour_coord), 3):
      coord.append((contour_coord[i], contour_coord[i + 1], contour_coord[i + 2]))
        
img_ID = contour_dataset.ContourImageSequence[0].ReferencedSOPInstanceUID
img = dicom.read_file(path + 'MR.'+img_ID + '.dcm')
img_arr = img.pixel_array

x_spacing, y_spacing = float(img.PixelSpacing[0]), float(img.PixelSpacing[1])

 # this is the center of the upper left voxel
origin_x, origin_y, _ = img.ImagePositionPatient

    # y, x is how it's mapped
pixel_coords = [(np.ceil((y - origin_y) / y_spacing), np.ceil((x - origin_x) / x_spacing)) for x, y, _ in coord]
rows = []
cols = []
for (i, j) in list(set(pixel_coords)):
    rows.append(i)
    cols.append(j)

print(list(set(pixel_coords)))

########
#NOT WORKING !!!!!! 
contour_arr = csc_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int8, shape=(img_arr.shape[0],img_arr.shape[1])).toarray()
########


./Breast_MRI_cont/Joly/
RS.1.2.246.352.71.4.139189879485.219264.20180517150349.dcm
7

 
 Countours[0]= ----------------------------------------
(3006, 0016)  Contour Image Sequence   1 item(s) ---- 
   (0008, 1150) Referenced SOP Class UID            UI: MR Image Storage
   (0008, 1155) Referenced SOP Instance UID         UI: 1.2.840.113619.2.410.9231572.3304717.14769.1478086720.344
   ---------
(3006, 0042) Contour Geometric Type              CS: 'CLOSED_PLANAR'
(3006, 0046) Number of Contour Points            IS: '24'
(3006, 0050) Contour Data                        DS: ['-75.66', '-110.03', '10.44', '-77.11', '-110.22', '10.44', '-77.62', '-110.54', '10.44', '-78.55', '-111.45', '10.44', '-78.89', '-111.98', '10.44', '-79.07', '-113.43', '10.44', '-79.27', '-114.87', '10.44', '-80', '-115.87', '10.44', '-80.2', '-116.32', '10.44', '-80', '-116.98', '10.44', '-79.76', '-117.77', '10.44', '-79.65', '-119.21', '10.44', '-78.55', '-120.23', '10.44', '-77.11', '-120.26', '10.44', '-76.07

ValueError: negative row index found

In [71]:
def coord2pixels(contour_dataset, path):
    ""
    contour_coord = contour_dataset.ContourData
    # x, y, z coordinates of the contour in mm
    coord = []
    for i in range(0, len(contour_coord), 3):
        coord.append((contour_coord[i], contour_coord[i + 1], contour_coord[i + 2]))

    # extract the image id corresponding to given countour
    # read that dicom file
    img_ID = contour_dataset.ContourImageSequence[0].ReferencedSOPInstanceUID
    img = dicom.read_file(path + img_ID + '.dcm')
    img_arr = img.pixel_array

    # physical distance between the center of each pixel
    x_spacing, y_spacing = float(img.PixelSpacing[0]), float(img.PixelSpacing[1])

    # this is the center of the upper left voxel
    origin_x, origin_y, _ = img.ImagePositionPatient

    # y, x is how it's mapped
    pixel_coords = [(np.ceil((y - origin_y) / y_spacing), np.ceil((x - origin_x) / x_spacing)) for x, y, _ in coord]

    # get contour data for the image
    rows = []
    cols = []
    for i, j in list(set(pixel_coords)):
        rows.append(i)
        cols.append(j)
    contour_arr = csc_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int8, shape=(img_arr.shape[0],img_arr.shape[1])).toarray()

    return img_arr, contour_arr, img_ID


(3006, 0082) Observation Number                  IS: '1'
(3006, 0084) Referenced ROI Number               IS: '1'
(3006, 0085) ROI Observation Label               SH: 'Organ'
(3006, 00a4) RT ROI Interpreted Type             CS: 'ORGAN'
(3006, 00a6) ROI Interpreter                     PN: ''

In [72]:
struct_f = dicom.dcmread(path+'RS.1.2.246.352.71.4.139189879485.219264.20180517150349.dcm')
print(struct_f)


(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0012) Instance Creation Date              DA: '20180529'
(0008, 0013) Instance Creation Time              TM: '141321.265000'
(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.4.139189879485.219264.20180517150349
(0008, 0020) Study Date                          DA: '20161102'
(0008, 0030) Study Time                          TM: '113929'
(0008, 0050) Accession Number                    SH: 'RA2016340454'
(0008, 0060) Modality                            CS: 'RTSTRUCT'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'TREMBLAY, FRANCINE'
(0008, 1010) Station Name                        SH: 'WNETAP170'
(0008, 1030) Study Description                   LO: 'MRI BREAST C- C+ BILATERAL -SE'
(0008, 103e) Series Description                  LO: 'ARI